In [ ]:
from dense import Dense
from activations import *
import numpy as np
from losses import mse, mse_prime

import bisect
from sklearn import preprocessing
import os
import pandas as pd
from parameters import *
import copy
import regex as re
import matplotlib.pyplot as plt

In [ ]:

os.chdir('C:\\Users\\91993\\Desktop\\chaos\\')
home_path = os.getcwd()
saved_path = home_path
saved_path



In [ ]:
initialization_list = [i for i in range(1,10)]

In [ ]:
def visualize_weights(network):
    for items in network:
        if items.__class__.__name__ == 'Dense':
            print(items.weights)
            print(items.bias)


def mask_weights(network,mask_list,weights_per_layer):
    if len(mask_list)!=0:
        for mask in mask_list:
            index = bisect.bisect_left(weights_per_layer,mask)
            shape_ = network[2*index].weights.shape


            if index!=0:
                mask = mask - weights_per_layer[index-1]


            network[2*index].weights[(mask-1)//shape_[1]][(mask-1)%shape_[1]] = 0




def predict(network, input,store_weights=False,list_w=[],list_b=[]):
    output = input
    for layer in network:
        if layer.__class__.__name__ =='Dense' and store_weights==True:
            output = layer.forward(output)
            list_w.append(np.copy(layer.weights))
            list_b.append(np.copy(layer.bias))
        else :
            output = layer.forward(output)


    return output

def get_accuracy(network,X_t,Y_t):
    correct = 0
    for x, y in zip(X_t, Y_t):
        output = predict(network, x,store_weights=False)
        if np.argmax(output)==np.argmax(y):
            correct +=1

    return correct

def train(network, loss, loss_prime, x_train, y_train,x_test,y_test, epochs = 1000, learning_rate = 0.01, verbose = True,store_weights=False,mask_weight=False,mask_list=[],weights_per_layer=[],list_w=[],list_b=[]):
    test_acc = []
    train_acc = []
    for e in range(epochs):
        error = 0
        for x, y in zip(x_train, y_train):
            # forward
            if mask_weight:
                mask_weights(network,mask_list=mask_list,weights_per_layer=weights_per_layer)
                
            if store_weights :
                output = predict(network, x,store_weights=store_weights,list_w=list_w,list_b=list_b)
            else:
                output = predict(network,x,store_weights=False)
            # print(output)
            # print(y)
            # print("")
            # error
            error += loss(y, output)

            # backward
            grad = loss_prime(y, output)
            for layer in reversed(network):
                grad = layer.backward(grad, learning_rate)

            

            if store_weights:
                train_accuracy = get_accuracy(network,x_train,y_train)
                accuracy = get_accuracy(network,x_test,y_test)
                test_acc.append(accuracy/x_test.shape[0])
                train_acc.append(train_accuracy/x_train.shape[0])

        if store_weights!=True:
            train_accuracy = get_accuracy(network,x_train,y_train)
            accuracy = get_accuracy(network,x_test,y_test)
            test_acc.append(accuracy/x_test.shape[0])
            train_acc.append(train_accuracy/x_train.shape[0])
                
        error /= len(x_train)
        if verbose:
            print(f"{e + 1}/{epochs}, error={error}")
        

    return train_acc,test_acc

def visualize(train_acc,test_acc):
    plt.plot(train_acc[:])
    plt.plot(test_acc[:])
    plt.legend(['train','test'])
    plt.show()

In [ ]:
network_list = []

In [ ]:
for init in initialization_list:
    home_path = saved_path
    os.chdir(home_path)
    path = home_path+'\\NN-RESULTS-FINAL\\datasets'
    datasets_path = os.path.join(path,"data")
    init_path = os.path.join(path,"initializations")
    os.chdir(datasets_path)


    datasets = os.listdir()

    #-----------------------------control------------------------------#
    data_curr = "binary_sonar"
    initialization = init
    new_init = True
    #------------------------------------------------------------------#


    #----------------------------------------------------------------------------------------#
    init_path = os.path.join(path,"initializations")
    path_initializations = f'{init_path}\\{data_curr}'
    try:
        list_init = os.listdir(path_initializations)
    except:
        os.mkdir(path_initializations)
        list_init = os.listdir(path_initializations)

    numbers = []

    print(list_init)

    for items in list_init:
        if(re.search(f'{data_curr}',items)):
            str_temp = items.split("-")[1]
            numbers.append(int(str_temp.split(".")[0]))




    if numbers.__contains__(initialization) and new_init:
        new_init = False
        print("initialization already exists")
    else:
        if len(numbers)==0:
            initialization = 1
        else:
            initialization = numbers[-1]+1

    #====================Data and Network Initialization======================================#
    params = parameter_file(data_curr,path,initialization,new_init)

    lr = params['learning_rate'] 
    epochs = params['num_epochs'] 
    loss = globals()[params['loss']]
    loss_prime = globals()[params['loss']+'_prime']
    network = params['network']
    network_list.append(network)
    start_weights = params['init']

    X_train,Y_train,X_test,Y_test = params['data']
    #=========================================================================================#
    if new_init:
        init_path = os.path.join(path,"initializations")
        try:
            os.mkdir(f'{init_path}\\{data_curr}')

        except:
            print("")

        path_new_init = f'{init_path}\\{data_curr}\\{data_curr}-{initialization}.npy'
        np.save(path_new_init,start_weights)

    weights_per_layer = []

    for items in network:
        if items.__class__.__name__ == 'Dense':
            weights_per_layer.append(items.weights.shape[0]*items.weights.shape[1])


    sum_ = weights_per_layer[0]
    for i in range(1,len(weights_per_layer)):
        sum_ += weights_per_layer[i]
        weights_per_layer[i] = sum_

In [ ]:
df_train_sparse = {f'{init}':[]for init in initialization_list}
df_test_sparse = {f'{init}':[]for init in initialization_list}
df_train_epoch = {f'{init}':[]for init in initialization_list}
df_test_epoch = {f'{init}':[]for init in initialization_list}

In [ ]:
visualize_weights(network_list[8])

In [ ]:

network = np.copy(network_list[8])
initialization = 1
# train_sparse, test_sparse = train(network, loss,loss_prime,X_train,Y_train,X_test,Y_test,epochs=epochs,learning_rate=lr,verbose=False,store_weights=False,mask_weight=True,mask_list=mask_list,weights_per_layer=weights_per_layer)
# network = initialize_Weights(network,params['init'])
train_acc, test_acc = train(network, loss,loss_prime,X_train,Y_train,X_test,Y_test,epochs=50,learning_rate=0.2,verbose=False,store_weights=False,mask_weight=False,mask_list=[],weights_per_layer=weights_per_layer)
# df_test_sparse[f'{initialization}'] = test_sparse
df_test_epoch[f'{initialization}'] = test_acc

# df_train_sparse[f'{initialization}'] = train_sparse
df_train_epoch[f'{initialization}'] = train_acc

In [ ]:
visualize(df_train_epoch[f'{initialization}'],df_test_epoch[f'{initialization}'])

In [ ]:
visualize(df_train_epoch[f'{initialization}'],df_test_epoch[f'{initialization}'])

In [ ]:
visualize(df_train_epoch[f'{initialization}'],df_test_epoch[f'{initialization}'])

In [ ]:
'''
testing for pytorch custom model 

'''

import numpy as np
import pandas as pd 
import torch
import torch.nn as nn
import torch.nn.functional as F 
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms 
import matplotlib.pyplot as plt
import csv


device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


class MNIST(nn.Module):
    def __init__(self,input_size, num_classes):
        super(MNIST,self).__init__()
        self.fc1 = nn.Linear(input_size,15)
        self.fc2 = nn.Linear(15,30)
        self.fc3 = nn.Linear(30,10)
        self.fc4 = nn.Linear(10,num_classes)
        self.softmax = nn.Softmax(dim=1)
        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()

    def forward(self,x):
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.relu(x)
        x = self.fc4(x)
        x = self.softmax(x)
        return x 

    def mask(self,model_tensor): 
        self.fc1.weight = torch.nn.parameter.Parameter(model_tensor['fc1']*self.fc1.weight)
        self.fc2.weight = torch.nn.parameter.Parameter(model_tensor['fc2']*self.fc2.weight)
        self.fc3.weight = torch.nn.parameter.Parameter(model_tensor['fc3']*self.fc3.weight)
        self.fc4.weight = torch.nn.parameter.Parameter(model_tensor['fc4']*self.fc4.weight)

class MNISTDataset(Dataset):
    def __init__(self, datapath,train,size):
        self.data = pd.read_csv(datapath)
        arr_data = np.array(self.data,dtype=np.float64)
        
        if train==True:
            arr_data = arr_data[:int(size*len(arr_data))]
        else : 
            arr_data = arr_data[int(size*len(arr_data)):]
        self.arr_data = (arr_data)
        self.x = torch.from_numpy(self.arr_data[:,1:])
        self.x = self.x.float()
        self.y = torch.from_numpy(self.arr_data[:,0])
        self.y = self.y.float()
       
    def __len__(self):
        return len(self.arr_data)

    def __getitem__(self,index):
        return self.x[index], F.one_hot(self.y[index].long(),10)


def model_summary(model):
    """
    get each layer weights, keys and shapes
    """
    assert isinstance(model,nn.Module)==True

    model_keys = []
    model_shapes = []
    for name, params in model.named_parameters():
        if name.split(".")[1]=="weight":
            model_keys.append(name.split(".")[0])
            model_shapes.append(params.shape[0]*params.shape[1])

    return model_keys,model_shapes



def get_weights(model:nn.Module,model_dict):
    """
    check whether model is instance of nn.Module
    """
    
    
    for name, param in model.named_parameters():
        if name.split(".")[1]=="weight":
            index = name.split(".")[0]
            temp_tensor = param.clone()
            temp_weights = temp_tensor.detach().cpu().numpy()
            temp_weights = temp_weights.reshape(-1,)
            
            model_dict[f'{index}'] = np.vstack((model_dict[f'{index}'],temp_weights))

    return model_dict




def save_weights(model:nn.Module,name):
    """
    check whether model is instance of nn.Module
    """
    W = []
    for layer, param in model.named_parameters():
        if layer.split(".")[1]=="weight":
            # index = name.split(".")[0]
            temp_tensor = param.clone()
            temp_weights = temp_tensor.detach().cpu().numpy()
            temp_weights = temp_weights.reshape(-1)
            W.extend(temp_weights.tolist())

    with open(name,'a',newline='') as file:
        writer = csv.writer(file, delimiter=',')
        writer.writerow(W)
    file.close()
    

    



def get_accuracy(model:nn.Module, name,dataloader,device,save_weights = False):
    model.eval()
    n_correct = 0
    n_samples = 0
    with torch.no_grad():
        for data,targets in dataloader:
            data = data.to(device)
            
            targets = targets.to(device)
            outputs = model(data)
            predictions = torch.argmax(outputs,1)
            targets = torch.argmax(targets,1)
            n_samples += targets.shape[0]
            n_correct += (predictions == targets).sum().tolist()

    acc = 100*n_correct/n_samples
    if save_weights:
        with open(name,'a',newline='') as file:
            writer = csv.writer(file, delimiter=',')
            writer.writerow([acc])
        file.close()
    else:
        return acc



In [1]:
import os


## THE PROFILER CELL

In [24]:
from torch.profiler import profile, record_function, ProfilerActivity
from torch_custom import *
import tqdm 


from torch_custom import *
import tqdm 

# start from 1
initialization = 1
torch.manual_seed(initialization)
np.random.seed(initialization)


# initialization = 5 seed = 0

name = f"weights{initialization}.csv"
train_acc = f"train_accuracies{initialization}.csv"
test_acc = f"test_accuracies{initialization}.csv"

datapath = r"mnist.csv"

"""
MODEL INTITIALIZATION AND SETTING UP THE HYPERPARAMETERS

"""

model = MNIST(784,10).to(device) #change the model definition from the torch_custom file to test on other datasets
train_size = 0.7
datasets = MNISTDataset(datapath,train=True,size=train_size) #change the dataset class from the torch_custom file
test_datasets = MNISTDataset(datapath,train=False,size=train_size)
batch_size = 128
dataloader = DataLoader(dataset=datasets,batch_size=batch_size,shuffle=True)
testloader = DataLoader(dataset=test_datasets,batch_size=batch_size)

num_epochs = 50
learning_rate = 0.5

n_total_steps = len(dataloader)
print(n_total_steps)

torch.save(model.state_dict(), f"model_{initialization}.pth")

criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)


w_num = 0
model_keys, model_shapes = model_summary(model)
for shapes in model_shapes:
    w_num += shapes             #shapes[0]*shapes[1]
#weights = np.zeros(shape=(n_total_steps*num_epochs,w_num))
print(n_total_steps*num_epochs,w_num)

acc = []
for i in range(0,25):
    r = 45*(i+1)+5
    acc.append(r)


j = 0

"""
TRAINING AND VALIDATION LOOPS 
"""
prof = torch.profiler.profile(
        schedule=torch.profiler.schedule(wait=1, warmup=1, active=3, repeat=2),
        on_trace_ready=torch.profiler.tensorboard_trace_handler('/log/torch_custom'),
        record_shapes=True,profile_memory=True,
        with_stack=True)
prof.start()

for epoch in range(num_epochs):

 
    model.train()
    for i,(data,targets) in enumerate(dataloader):
#       weights[j] = get_weights(model=model)
        j+=1
        data = data.to(device)
        temp_targets = targets
        targets = targets.float()

        targets = targets.to(device)
   
        outputs = model(data)
        loss = criterion(outputs,targets)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if(i+1)%200==0:
            print(f'epoch {epoch+1} / {num_epochs}, step {i+1}/{n_total_steps}, loss = {loss.item():.4f}')
#        if epoch*n_total_steps+i in acc:
#            get_accuracy(model=model,name =train_acc,dataloader=dataloader,device=device,save_weights=True)
#            get_accuracy(model=model,name = test_acc,dataloader=testloader,device=device,save_weights=True)
    print(epoch)
    print("train",get_accuracy(model=model,name =train_acc,device=device,dataloader=dataloader))
    print("test",get_accuracy(model=model,name = test_acc,device=device,dataloader=testloader))


prof.stop()
#np.savetxt(f"weights{initialization}.csv",weights,delimiter=',')
#weights = pd.DataFrame(weights)
#weights.to_csv(f"weights{initialization}.csv",header=False,index=False)




230
11500 41000
epoch 1 / 50, step 200/230, loss = 0.0882
0
train 20.269396918262526
test 20.33965558289025
epoch 2 / 50, step 200/230, loss = 0.0873
1
train 24.796761794618863
test 25.085310689627807
epoch 3 / 50, step 200/230, loss = 0.0799
2
train 36.576074016122995
test 37.012935481311004
epoch 4 / 50, step 200/230, loss = 0.0801
3
train 46.675056974727035
test 47.20260296801841
epoch 5 / 50, step 200/230, loss = 0.0753
4
train 48.60709547943808
test 49.09927783509245
epoch 6 / 50, step 200/230, loss = 0.0686
5
train 49.341814347426784
test 49.77382747401
epoch 7 / 50, step 200/230, loss = 0.0613
6
train 54.18211503792646
test 54.55122609316721
epoch 8 / 50, step 200/230, loss = 0.0538
7
train 62.845675022959966
test 62.96325688437425
epoch 9 / 50, step 200/230, loss = 0.0477
8
train 73.13173917480187
test 72.67677168478693
epoch 10 / 50, step 200/230, loss = 0.0422
9
train 78.56049525494065
test 78.0176176493929
epoch 11 / 50, step 200/230, loss = 0.0357
10
train 82.22048368992142

In [41]:

inputs = torch.randn(1, 784)
with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
        model(inputs)

print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))

----------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                  Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
----------------------  ------------  ------------  ------------  ------------  ------------  ------------  
       model_inference        17.95%     337.000us       100.00%       1.877ms       1.877ms             1  
          aten::linear         1.60%      30.000us        78.96%       1.482ms     494.000us             3  
           aten::addmm        68.30%       1.282ms        71.34%       1.339ms     446.333us             3  
               aten::t         3.52%      66.000us         6.02%     113.000us      37.667us             3  
       aten::transpose         1.70%      32.000us         2.50%      47.000us      15.667us             3  
         aten::sigmoid         2.29%      43.000us         2.29%      43.000us      21.500us             2  
           aten::co

STAGE:2023-05-04 12:43:07 23102:23102 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2023-05-04 12:43:07 23102:23102 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-05-04 12:43:07 23102:23102 ActivityProfilerController.cpp:321] Completed Stage: Post Processing


In [42]:
print(prof.key_averages().table(sort_by="self_cpu_memory_usage", row_limit=10))

----------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                  Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
----------------------  ------------  ------------  ------------  ------------  ------------  ------------  
       model_inference        17.95%     337.000us       100.00%       1.877ms       1.877ms             1  
          aten::linear         1.60%      30.000us        78.96%       1.482ms     494.000us             3  
               aten::t         3.52%      66.000us         6.02%     113.000us      37.667us             3  
       aten::transpose         1.70%      32.000us         2.50%      47.000us      15.667us             3  
      aten::as_strided         0.85%      16.000us         0.85%      16.000us       2.667us             6  
           aten::addmm        68.30%       1.282ms        71.34%       1.339ms     446.333us             3  
          aten::exp